In [ ]:
# ────────────────────────────────────────────────────────────────
# 📦  Cell 1 – one-time setup
#     (installs Crawl4AI, Playwright + its browsers, and helpers)
# ────────────────────────────────────────────────────────────────
%pip install --quiet "crawl4ai[playwright]" requests tqdm nest_asyncio

import subprocess, sys, nest_asyncio
nest_asyncio.apply()

# install headless Chromium etc. (only the first time on a machine)
subprocess.run(
    [sys.executable, "-m", "playwright", "install", "--with-deps"],
    check=True,
)


In [ ]:
# 🔧 install the missing converter
%pip install --quiet markdownify beautifulsoup4


In [ ]:
!pip install crawl4ai markdownify tqdm beautifulsoup4 pandas

In [ ]:
# In a new cell (or un-comment the install cell already in the notebook)
!pip install async-timeout aiohttp
# If you haven’t already, also make sure the others are present:
!pip install crawl4ai markdownify beautifulsoup4 tqdm playwright
!playwright install


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  scrape_to_markdown + example call                           ║
# ╚══════════════════════════════════════════════════════════════╝
import asyncio, os, re
from pathlib import Path
from urllib.parse import urljoin, urlsplit, quote

import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from tqdm.auto import tqdm

from crawl4ai import AsyncWebCrawler, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
# If the page does NOT need JavaScript, uncomment the line below for faster runs
# from crawl4ai.async_crawler_strategy import AsyncHTTPCrawlerStrategy


# ─────────────────────────── noise filters ─────────────────────
NOISE_RE   = re.compile(
    r"(cookie|consent|banner|nav|menu|breadcrumb|footer|sidebar|masthead|navbar|"
    r"pager|pagination|social|share)",
    re.I,
)
STRIP_TAGS = {"svg", "style", "script"}

# ─────────────────────────── helpers ───────────────────────────
def _safe_img_name(url: str, idx: int) -> str:
    ext = os.path.splitext(urlsplit(url).path)[1]
    return f"{idx:03d}{ext or '.jpg'}"

def _best_wrapper(soup: BeautifulSoup):
    for sel in ("main[id*='Main-Content' i]", "main", "article", "#content", ".content"):
        node = soup.select_one(sel)
        if node:
            return node
    divs = sorted(soup.find_all("div"),
                  key=lambda d: len(d.get_text(" ", strip=True)),
                  reverse=True)
    return divs[0] if divs else soup.body or soup

def _strip_noise(node: BeautifulSoup):
    for tag in node.find_all(["header", "nav", "footer", "aside"]):
        tag.decompose()
    for tag in node.find_all(
        lambda t: (t.has_attr("id") and NOISE_RE.search(t["id"]))
        or (t.has_attr("class") and any(NOISE_RE.search(c) for c in t["class"]))
    ):
        tag.decompose()
    for tag in node.find_all(STRIP_TAGS):
        tag.decompose()

# ─────────────────────────── main coroutine ────────────────────
async def scrape_to_markdown(
    url: str,
    *,
    out_md: str | Path = "output.md",
    assets_dir: str | Path = "assets",
    css_selector: str | None = None,
    wait_for_images: bool = True,
) -> Path:
    """
    Crawl `url`, clean it up, download images, and write a Markdown file
    that starts with front-matter containing `page_url`.
    """
    out_md     = Path(out_md)
    assets_dir = Path(assets_dir)
    BASE       = f"{urlsplit(url).scheme}://{urlsplit(url).netloc}"

    run_cfg = CrawlerRunConfig(
        css_selector="body",
        wait_for_images=wait_for_images,
        remove_overlay_elements=True,
        js_code="""(() => {
            [...document.querySelectorAll('button,input[type="button"]')]
              .filter(b => /accept|agree|ok/i.test(b.textContent))
              .forEach(b => b.click());
        })();""",
        markdown_generator=DefaultMarkdownGenerator(
            options={"ignore_links": False, "ignore_images": False}
        ),
    )

    # async with AsyncWebCrawler(crawler_strategy=AsyncHTTPCrawlerStrategy()) as crawler:
    async with AsyncWebCrawler() as crawler:  # Playwright default
        result = await crawler.arun(url, config=run_cfg)

    if not result.success:
        raise RuntimeError(result.error_message)

    soup  = BeautifulSoup(result.html, "html.parser")
    main  = soup.select_one(css_selector) if css_selector else _best_wrapper(soup)
    html  = BeautifulSoup(str(main), "html.parser")

    _strip_noise(html)

    # ---- rewrite links ---------------------------------------------------------
    for a in html.find_all("a", href=True):
        href = a["href"].strip()
        if not href or href.startswith(("javascript:", "#")):
            a.unwrap()
            continue
        if href.startswith("//"):
            href = "https:" + href
        elif not href.startswith(("http://", "https://", "mailto:", "tel:")):
            href = urljoin(BASE, href)
        a["href"] = quote(href, safe="/#:?=&;+@-._~%")

    # ---- download images -------------------------------------------------------
    assets_dir.mkdir(exist_ok=True)
    sess = requests.Session()
    for idx, img in enumerate(tqdm(html.find_all("img"), desc="images", unit="img")):
        src = img.get("data-src") or img.get("src")
        if not src or src.startswith("data:"):
            continue
        full = urljoin(url, src)
        try:
            r = sess.get(full, timeout=30)
            r.raise_for_status()
            fname = _safe_img_name(full, idx)
            (assets_dir / fname).write_bytes(r.content)
            img["src"] = f"{assets_dir}/{fname}"
        except Exception as e:
            print(f"⚠ cannot fetch {full}: {e}")

    # ---- HTML → Markdown -------------------------------------------------------
    body_md = md(str(html), heading_style="ATX", strip=["script", "style"])

    # ---- prepend YAML front-matter --------------------------------------------
    front = f"---\npage_url: {url}\n---\n\n"
    out_md.write_text(front + body_md, encoding="utf-8")
    print(f"✅  Wrote {out_md} and saved {len(list(assets_dir.glob('*')))} image(s).")
    return out_md


In [ ]:
# ──────────────────────── example call ───────────────────────
url = "https://www.essex.ac.uk/student/professional-services/occupational-health-team"

await scrape_to_markdown(
    url,
    out_md="output20.md",   # output Markdown filename
    assets_dir="assets",    # folder for downloaded images
    # css_selector="#main", # uncomment if you need to force a wrapper
)


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  md_to_json – read Markdown w/ front-matter   + demo call    ║
# ╚══════════════════════════════════════════════════════════════╝
import json, re
from pathlib import Path
from urllib.parse import urljoin, urlparse
from markdown import markdown               # pip install markdown
from bs4 import BeautifulSoup               # pip install beautifulsoup4

FRONT_RE = re.compile(r"^---\s*$")          # detect YAML fence

def md_to_json(md_path: str | Path, *, save: bool = True) -> Path | dict:
    """
    Convert a Markdown file (produced by scrape_to_markdown) into structured
    JSON and optionally save it next to the .md.

    Expected Markdown header:
        ---
        page_url: https://example.com/…
        ---

    Output JSON format:
        {
          "page_url": "...",
          "internal_links": [ "...", ... ],
          "sections": [ {title, level, content, children}, ... ]
        }
    """
    md_path = Path(md_path)
    if not md_path.is_file():
        raise FileNotFoundError(md_path)

    # ── 1. split front-matter & body ───────────────────────────
    lines = md_path.read_text(encoding="utf-8").splitlines()
    if not (lines and FRONT_RE.match(lines[0])):
        raise ValueError("Markdown missing expected '---' front-matter fence.")
    end_idx = next(i for i, ln in enumerate(lines[1:], 1) if FRONT_RE.match(ln))
    front = lines[1:end_idx]                          # between the fences
    body_md = "\n".join(lines[end_idx + 1:])

    # extract page_url from front-matter
    page_url = None
    for ln in front:
        if ln.lower().startswith("page_url:"):
            page_url = ln.split(":", 1)[1].strip()
            break
    if not page_url:
        raise ValueError("page_url not found in front-matter.")

    parsed_page = urlparse(page_url)
    domain      = parsed_page.netloc.lower()

    # ── 2. Markdown → HTML → soup ──────────────────────────────
    html = markdown(body_md, extensions=["extra", "tables", "fenced_code"])
    soup = BeautifulSoup(html, "html.parser")

    def abs_url(u: str) -> str:
        if u.startswith(("http://", "https://", "mailto:", "tel:")):
            return u
        if u.startswith("//"):
            return f"{parsed_page.scheme}:{u}"
        return urljoin(page_url, u)

    # ── 3. collect internal links (same domain) ───────────────
    internal = sorted(
        {
            abs_url(a["href"].strip())
            for a in soup.find_all("a", href=True)
            if urlparse(abs_url(a["href"].strip())).netloc.lower() == domain
        }
    )

    # ── 4. build nested sections ───────────────────────────────
    root, stack = [], []
    for node in soup.recursiveChildGenerator():
        if isinstance(node, str):
            continue
        if node.name in {"h1", "h2", "h3", "h4", "h5", "h6"}:
            lvl = int(node.name[1])
            sec = {"title": node.get_text(" ", strip=True),
                   "level": lvl,
                   "content": [],
                   "children": []}
            while stack and stack[-1]["level"] >= lvl:
                stack.pop()
            (stack[-1]["children"] if stack else root).append(sec)
            stack.append(sec)
        elif stack:
            sec = stack[-1]
            if node.name == "p":
                txt = node.get_text(" ", strip=True)
                if txt:
                    sec["content"].append({"type": "paragraph", "text": txt})
            elif node.name in {"ul", "ol"}:
                items = [
                    li.get_text(" ", strip=True)
                    for li in node.find_all("li", recursive=False)
                ]
                sec["content"].append({"type": "list",
                                       "ordered": node.name == "ol",
                                       "items": items})
            elif node.name == "img":
                sec["content"].append({"type": "image",
                                       "url": abs_url(node.get("src", "")),
                                       "alt": node.get("alt", "")})
            elif node.name == "a":
                sec["content"].append({"type": "link",
                                       "url": abs_url(node.get("href", "")),
                                       "text": node.get_text(" ", strip=True)})

    kb_json = {"page_url": page_url,
               "internal_links": internal,
               "sections": root}

    if save:
        out_path = md_path.with_suffix(".json")
        out_path.write_text(json.dumps(kb_json, indent=2, ensure_ascii=False),
                            encoding="utf-8")
        print(f"✅  Wrote {out_path}")
        return out_path
    return kb_json



In [ ]:
# ────────────────────────── demo call ─────────────────────────
# Convert the Markdown file produced earlier into JSON
md_to_json("output20.md")        # writes output15.json next to the .md



In [2]:
from urllib.parse import urlparse, unquote
import pandas as pd

df = pd.read_csv("essex_url_hierarchy_final.csv")

# ── Grab every URL whose level_1 == "courses" ──────────────────────────
course_urls = df.loc[df["level_1"] == "courses", "url"].dropna()

# ── Build slug-variant → full-URL dictionary ───────────────────────────
course_dict = {}
for u in course_urls:
    parts   = unquote(urlparse(u).path).rstrip("/").split("/")
    slug    = parts[-1]          # final bit, e.g. 'BSc-Accounting-and-Finance'
    variant = parts[-2]          # the one just before it, e.g. '1', '2', ...
    
    key = f"{slug}-{variant}" if variant.isdigit() else slug
    course_dict[key] = u

# ── Sanity check ───────────────────────────────────────────────────────
print(len(course_dict))          # → 685
print(list(course_dict.items())[:5])


685
[('BSc-Accounting-1', 'https://www.essex.ac.uk/courses/UG00001/1/BSc-Accounting'), ('MSc-Accounting-1', 'https://www.essex.ac.uk/courses/PG00425/1/MSc-Accounting'), ('PhD-Accounting-2', 'https://www.essex.ac.uk/courses/PR00854/2/PhD-Accounting'), ('BSc-Accounting-and-Finance-1', 'https://www.essex.ac.uk/courses/UG00002/1/BSc-Accounting-and-Finance'), ('PhD-Accounting-and-Finance-5', 'https://www.essex.ac.uk/courses/PR00855/5/PhD-Accounting-and-Finance')]


In [4]:
%pip install -q pandas tqdm markdownify beautifulsoup4 requests crawl4ai nest_asyncio
!python -m playwright install --with-deps


Note: you may need to restart the kernel to use updated packages.


In [6]:
import asyncio, os, re
from pathlib import Path
from urllib.parse import urljoin, urlsplit, urlparse, quote, unquote

import nest_asyncio, pandas as pd, requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from tqdm.notebook import tqdm

nest_asyncio.apply()  # let us use 'await' at the notebook top-level

from crawl4ai import AsyncWebCrawler, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
# If pages don't need JavaScript, you could speed things up with:
# from crawl4ai.async_crawler_strategy import AsyncHTTPCrawlerStrategy

NOISE_RE   = re.compile(
    r"(cookie|consent|banner|nav|menu|breadcrumb|footer|sidebar|masthead|navbar|"
    r"pager|pagination|social|share)",
    re.I,
)
STRIP_TAGS = {"svg", "style", "script"}

def _safe_img_name(url: str, idx: int) -> str:
    ext = os.path.splitext(urlsplit(url).path)[1]
    return f"{idx:03d}{ext or '.jpg'}"

def _best_wrapper(soup: BeautifulSoup):
    for sel in ("main[id*='Main-Content' i]", "main", "article", "#content", ".content"):
        if (node := soup.select_one(sel)):
            return node
    divs = sorted(soup.find_all("div"),
                  key=lambda d: len(d.get_text(" ", strip=True)),
                  reverse=True)
    return divs[0] if divs else soup.body or soup

def _strip_noise(node: BeautifulSoup):
    for tag in node.find_all(["header", "nav", "footer", "aside"]):
        tag.decompose()
    for tag in node.find_all(
        lambda t: (t.has_attr("id") and NOISE_RE.search(t["id"]))
        or (t.has_attr("class") and any(NOISE_RE.search(c) for c in t["class"]))
    ):
        tag.decompose()
    for tag in node.find_all(STRIP_TAGS):
        tag.decompose()

async def scrape_to_markdown(
    url: str,
    *,
    out_md: Path,
    assets_dir: Path,
    wait_for_images: bool = True,
):
    """Fetch `url`, strip boilerplate, download images, save Markdown."""
    BASE = f"{urlsplit(url).scheme}://{urlsplit(url).netloc}"

    run_cfg = CrawlerRunConfig(
        css_selector="body",
        wait_for_images=wait_for_images,
        remove_overlay_elements=True,
        js_code="""(() => {
            [...document.querySelectorAll('button,input[type="button"]')]
              .filter(b => /accept|agree|ok/i.test(b.textContent))
              .forEach(b => b.click());
        })();""",
        markdown_generator=DefaultMarkdownGenerator(
            options={"ignore_links": False, "ignore_images": False}
        ),
    )

    # async with AsyncWebCrawler(crawler_strategy=AsyncHTTPCrawlerStrategy()) as crawler:
    async with AsyncWebCrawler() as crawler:  # Playwright (handles JS)
        result = await crawler.arun(url, config=run_cfg)
    if not result.success:
        raise RuntimeError(result.error_message)

    soup  = BeautifulSoup(result.html, "html.parser")
    main  = _best_wrapper(soup)
    html  = BeautifulSoup(str(main), "html.parser")
    _strip_noise(html)

    # rewrite links
    for a in html.find_all("a", href=True):
        href = a["href"].strip()
        if not href or href.startswith(("javascript:", "#")):
            a.unwrap();  continue
        if href.startswith("//"):
            href = "https:" + href
        elif not href.startswith(("http://", "https://", "mailto:", "tel:")):
            href = urljoin(BASE, href)
        a["href"] = quote(href, safe="/#:?=&;+@-._~%")

    # download images
    assets_dir.mkdir(parents=True, exist_ok=True)
    sess = requests.Session()
    for idx, img in enumerate(tqdm(html.find_all("img"), desc=f"images:{out_md.stem}", leave=False)):
        src = img.get("data-src") or img.get("src")
        if not src or src.startswith("data:"):
            continue
        full = urljoin(url, src)
        try:
            r = sess.get(full, timeout=30);  r.raise_for_status()
            fname = _safe_img_name(full, idx)
            (assets_dir / fname).write_bytes(r.content)
            img["src"] = f"{assets_dir.name}/{fname}"
        except Exception as e:
            print(f"⚠ cannot fetch {full}: {e}")

    body_md = md(str(html), heading_style="ATX", strip=["script", "style"])
    front   = f"---\npage_url: {url}\n---\n\n"
    out_md.write_text(front + body_md, encoding="utf-8")


In [8]:
def build_course_dict(csv_path: str | Path) -> dict[str, str]:
    df = pd.read_csv(csv_path)
    course_urls = df.loc[df["level_1"] == "courses", "url"].dropna()

    course_dict: dict[str, str] = {}
    for u in course_urls:
        parts   = unquote(urlparse(u).path).rstrip("/").split("/")
        slug    = parts[-1]        # last segment
        variant = parts[-2]        # the "1/2/3" segment
        key = f"{slug}-{variant}" if variant.isdigit() else slug
        course_dict[key] = u
    return course_dict

course_dict = build_course_dict("essex_url_hierarchy_final.csv")
print(f"Total course pages: {len(course_dict)}")   # should be 685


Total course pages: 685


In [10]:
async def scrape_all(
    course_dict: dict[str, str],
    *,
    out_dir: str | Path = "course pages",
    concurrency: int = 8,
):
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True)

    sem   = asyncio.Semaphore(concurrency)
    prog  = tqdm(total=len(course_dict), unit="page")

    async def one_page(key: str, url: str):
        safe_key   = re.sub(r"[^\w\-]+", "-", key)
        md_path    = out_dir / f"{safe_key}.md"
        assets_dir = out_dir / safe_key
        try:
            async with sem:
                await scrape_to_markdown(
                    url,
                    out_md     = md_path,
                    assets_dir = assets_dir,
                )
        except Exception as e:
            print(f"⚠️  {key}: {e}")
        prog.update(1)

    await asyncio.gather(*(one_page(k, u) for k, u in course_dict.items()))
    prog.close()

# -- kick it off (just 'await' in a notebook) --
await scrape_all(course_dict, out_dir="course pages", concurrency=8)


  0%|          | 0/685 [00:00<?, ?page/s]

[INIT].... → Crawl4AI 0.6.2
[INIT].... → Crawl4AI 0.6.2
[INIT].... → Crawl4AI 0.6.2
[INIT].... → Crawl4AI 0.6.2
[INIT].... → Crawl4AI 0.6.2
[INIT].... → Crawl4AI 0.6.2
[INIT].... → Crawl4AI 0.6.2
[INIT].... → Crawl4AI 0.6.2
[SCRAPE].. ◆ Some images failed to load within timeout
[SCRAPE].. ◆ Some images failed to load within timeout
[SCRAPE].. ◆ Some images failed to load within timeout
[FETCH]... ↓ https://www.essex.ac.uk/courses/PR00854/1/PhD-Accounting                                             | ✓ | ⏱: 17.98s
[SCRAPE].. ◆ https://www.essex.ac.uk/courses/PR00854/1/PhD-Accounting                                             | ✓ | ⏱: 0.56s
[COMPLETE] ● https://www.essex.ac.uk/courses/PR00854/1/PhD-Accounting                                             | ✓ | ⏱: 18.55s


images:PhD-Accounting-1:   0%|          | 0/10 [00:00<?, ?it/s]

[FETCH]... ↓ https://www.essex.ac.uk/courses/UG00002/1/BSc-Accounting-and-Finance                                 | ✓ | ⏱: 20.18s
[SCRAPE].. ◆ https://www.essex.ac.uk/courses/UG00002/1/BSc-Accounting-and-Finance                                 | ✓ | ⏱: 0.23s
[COMPLETE] ● https://www.essex.ac.uk/courses/UG00002/1/BSc-Accounting-and-Finance                                 | ✓ | ⏱: 20.44s
[FETCH]... ↓ https://www.essex.ac.uk/courses/PR00854/2/PhD-Accounting                                             | ✓ | ⏱: 20.67s
[SCRAPE].. ◆ https://www.essex.ac.uk/courses/PR00854/2/PhD-Accounting                                             | ✓ | ⏱: 0.16s
[COMPLETE] ● https://www.essex.ac.uk/courses/PR00854/2/PhD-Accounting                                             | ✓ | ⏱: 20.88s
[FETCH]... ↓ https://www.essex.ac.uk/courses/UG00001/1/BSc-Accounting                                             | ✓ | ⏱: 20.84s
[SCRAPE].. ◆ https://www.essex.ac.uk/courses/UG00001/1/BSc-Accounting                       

images:BSc-Accounting-and-Finance-1:   0%|          | 0/10 [00:00<?, ?it/s]

...

[FETCH]... ↓ https://www.essex.ac.uk/courses/PG01071/1/MA-Translation-and-Professional-Practice                   | ✓ | ⏱: 71.21s
[SCRAPE].. ◆ https://www.essex.ac.uk/courses/PG01071/1/MA-Translation-and-Professional-Practice                   | ✓ | ⏱: 0.22s
[COMPLETE] ● https://www.essex.ac.uk/courses/PG01071/1/MA-Translation-and-Professional-Practice                   | ✓ | ⏱: 73.05s


images:MA-Advanced-Interpreting-with-Specialised-Translation-Chinese-English-1:   0%|          | 0/10 [00:00<?…

images:MA-Translation-and-Professional-Practice-1:   0%|          | 0/10 [00:00<?, ?it/s]

[FETCH]... ↓ https://www.essex.ac.uk/courses/PG00836/1/MA-Translation-Interpreting-and-Subtitling                 | ✓ | ⏱: 29.01s
[SCRAPE].. ◆ https://www.essex.ac.uk/courses/PG00836/1/MA-Translation-Interpreting-and-Subtitling                 | ✓ | ⏱: 0.23s
[COMPLETE] ● https://www.essex.ac.uk/courses/PG00836/1/MA-Translation-Interpreting-and-Subtitling                 | ✓ | ⏱: 30.78s


images:MA-Translation-Interpreting-and-Subtitling-1:   0%|          | 0/11 [00:00<?, ?it/s]